<a href="https://colab.research.google.com/github/sandeepkumar-84/DBS/blob/main/Research_ChatBot_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install nltk


In [12]:
pip install newspaper3k


In [13]:
pip install lxml_html_clean

In [21]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [43]:
from newspaper import Article
import random
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [44]:
nltk.download('punkt',quiet=True)

True

In [46]:
import PyPDF2

# extract text from single pdf file
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file.

    Args:
        pdf_path: The path to the PDF file.

    Returns:
        The extracted text as a string.
    """

    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        # Changed from pdf_reader.numPages to len(pdf_reader.pages)
        num_pages = len(pdf_reader.pages)
        text = ""

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    return text

# Example usage:
pdf_file_path = "/content/dbs-postgraduate-programmes.pdf"  # Replace with your PDF file path
extracted_text = extract_text_from_pdf(pdf_file_path)

print(extracted_text)

Postgraduate 
Programmes
Dublin Business School  Postgraduate Programmes www.dbs.ie
dbs.ieVisit our website: 
www.dbs.ie
Welcome
Dublin Business School (DBS) 
specialises in the provision of 
career-focused business and 
law education as well as the 
delivery of contemporary 
programmes in the areas of 
creative media, marketing, 
counselling & psychotherapy, 
humanities and psychology.Contents
Why Study for a Postgraduate Qualification?  2
Why Choose DBS? 3
DBS – A Short History 5
Recent News 6
DBS, QQI and the 
National Framework of Qualifications  8
Recognition and Accreditation 9
Career Services  10
Library and Information Services 12
Student Services  14
International Student Services  16
International Partnerships 18
Application Information and Procedures  20
DBS Professional School 90
Come Visit Us 91 
Psychology, Counselling  
and Creative Media
Higher Diploma in Arts 
in Psychology (PSI accredited) 23
Higher Diploma in Arts in Counselling 
and Psychotherapy 25
Master of Arts (

In [74]:
# extract combined texts from all the pdfs
import PyPDF2
import os

def extract_text_from_pdfs(pdf_dir):
    """
    Extracts text from all PDF files in a given directory and combines them.

    Args:
        pdf_dir: The path to the directory containing PDF files.

    Returns:
        A string containing the combined text from all PDFs.
    """

    all_text = ""  # Initialize an empty string to store all text

    # Check if the directory exists before trying to access it
    if not os.path.exists(pdf_dir):
        raise FileNotFoundError(f"The directory '{pdf_dir}' does not exist.")

    for filename in os.listdir(pdf_dir):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_dir, filename)
            with open(pdf_path, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                num_pages = len(pdf_reader.pages)
                for page_num in range(num_pages):
                    page = pdf_reader.pages[page_num]
                    all_text += page.extract_text()  # Append text to all_text

    return all_text

# Example usage:
# Ensure this path is correct and the directory exists
pdf_directory = r"/content/Research"
corpus = extract_text_from_pdfs(pdf_directory)  # Assign combined text to corpus

print(corpus)  # Print the combined text

StudentFAQGuideDublinBusinessSchoolLiveDocument
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Student
FAQ
Guide
Click
the
question
below
to
land
on
the
answer
you
need.
General
Queries
3
Will
exams
remain
online
for
the
duration
of
2023/2024?
3
Will
all
lectures
be
recorded
and
available
after
the
live
delivery?
3
Quality
Assurance
Handbook
(QAH)
3
How
do
I
upload
my
photo
for
the
student
card?
3
I
cannot
sit
an
exam/I
cannot
submit
my
assignment
on
time
4
What
grade
do
I
need
to
pass
a
module?
4
What
is
the
procedure
to
repeat
an
assignment
or
an
exam?
4
Where
can
I
find
my
timetable
for
my
classes?
4
How
do
I
enquire
about
changing
my
course?
5
I
want
to
change
or
deregister
a
module
5
Student
cards
for
new
students
5
Moodle
5
MS
Office
6
Letter
Requests
6
I
am
an
international
student
and
I
need
letters
for
my
visa
extension,
for
new 
accommodation,
and
to
open
a
new
bank
account.
Where
do


In [75]:
nltk.download('punkt_tab')

text = corpus
sentence_list = nltk.sent_tokenize(text)

print(sentence_list)

['StudentFAQGuideDublinBusinessSchoolLiveDocument\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _\nStudent\nFAQ\nGuide\nClick\nthe\nquestion\nbelow\nto\nland\non\nthe\nanswer\nyou\nneed.', 'General\nQueries\n3\nWill\nexams\nremain\nonline\nfor\nthe\nduration\nof\n2023/2024?', '3\nWill\nall\nlectures\nbe\nrecorded\nand\navailable\nafter\nthe\nlive\ndelivery?', '3\nQuality\nAssurance\nHandbook\n(QAH)\n3\nHow\ndo\nI\nupload\nmy\nphoto\nfor\nthe\nstudent\ncard?', '3\nI\ncannot\nsit\nan\nexam/I\ncannot\nsubmit\nmy\nassignment\non\ntime\n4\nWhat\ngrade\ndo\nI\nneed\nto\npass\na\nmodule?', '4\nWhat\nis\nthe\nprocedure\nto\nrepeat\nan\nassignment\nor\nan\nexam?', '4\nWhere\ncan\nI\nfind\nmy\ntimetable\nfor\nmy\nclasses?', '4\nHow\ndo\nI\nenquire\nabout\nchanging\nmy\ncourse?', '5\nI\nwant\nto\nchange\nor\nderegister\na\nmodule\n5\nStudent\ncards\nfor\nnew\nstudents\n5\nMoodle\n5\nMS\nOffice\n6\nLetter\

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [76]:
def greeting_response(text):
    text = text.lower()
    bot_greetings = ['howdy', 'hi', 'hello', 'hey', 'hola', 'namaste', 'bonjour', 'heita']
    user_greetings = ['hi', 'hey', 'hello', 'hola', 'greetings', 'namaste', 'bonjour', 'heita']

    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)

In [77]:
def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))

    x = list_var
    for i in range(length):
        for j in range(length):
          if x[list_index[i]] > x[list_index[j]]:
            temp = list_index[i]
            list_index[i] = list_index[j]
            list_index[j] = temp
    return list_index

In [78]:
def bot_response(user_input):
    user_input = user_input.lower()
    sentence_list.append(user_input)
    bot_response = ''
    cm = CountVectorizer().fit_transform(sentence_list)
    similarity_scores = cosine_similarity(cm[-1], cm)
    similarity_scores_list = similarity_scores.flatten()
    index = index_sort(similarity_scores_list)
    index = index[1:]
    response_flag = 0
    j = 0
    for i in range(len(index)):
        if similarity_scores_list[index[i]] > 0.0:
            bot_response = bot_response+' '+sentence_list[index[i]]
            response_flag = 1
            j = j+1
        if j > 2:
            break
    if response_flag == 0:
        bot_response = bot_response+' '+"I apologize, I don't understand."
    sentence_list.remove(user_input)
    return bot_response



In [79]:
print('DBS bot: I am your DBS bot. I will answer your queries about DBS. If you want to exit, type bye.')
exit_list = ['exit', 'see you later', 'bye', 'quit', 'break']
while(True):
    user_input = input()
    user_input = user_input.lower()
    if user_input in exit_list:
        print('DBS bot: Chat with you later!')
        break
    else:
        if greeting_response(user_input) != None:
            print('DBS bot: '+greeting_response(user_input))
        else:
            response = bot_response(user_input)
            print('DBS bot: '+ response)



DBS bot: I am your DBS bot. I will answer your queries about DBS. If you want to exit, type bye.
hi
DBS bot: bonjour
Will exams remain online for the duration of 2023/2024?
DBS bot:  General
Queries
3
Will
exams
remain
online
for
the
duration
of
2023/2024? 17
StudentFAQGuideDublinBusinessSchoolLiveDocument
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
General
Queries
Will
exams
remain
online
for
the
duration
of
2023/2024? Exams
will
continue
to
be
held
online
for
the
foreseeable
future,
including
Summer
exams.
what is QAH
DBS bot:  What
hours
is
the
library
open? 9
A
computer
is
not
working,
what
should
I
do? 10
What
hours
is
the
library
open?
Quality Assurance Handbook
DBS bot:  3
Quality
Assurance
Handbook
(QAH)
3
How
do
I
upload
my
photo
for
the
student
card? Quality
Assurance
Handbook
(QAH)
The
policies
and
procedures
that
govern
the
running
of
DBS
programmes
are
located
within
our
Quality

In [109]:
import pandas as pd
import re

# Assuming 'extracted_text' is a string containing the extracted text from PDF
# and you want to create a single-column DataFrame

# Convert the string to a list with a single element
extracted_text_list = [extracted_text]

# Create the DataFrame with the 'text' column
pdf_dataset = pd.DataFrame(extracted_text_list, columns=['text'])

# Define intent patterns (same as before)
intent_patterns = {
    "Admissions": [r"application", r"admission"],
    "Courses": [r"programmes?", r"course", r"diploma", r"qualification"],
    "Events": [r"events?", r"news"],
    "General Information": [r"welcome", r"about", r"history", r"services", r"contact"],
}

def classify_intent(text):
    # Initialize intent to None
    intent = None

    # Check for matches in each intent category
    for intent_category, patterns in intent_patterns.items():
        for pattern in patterns:
            if re.search(pattern, text, re.IGNORECASE):
                intent = intent_category  # Assign intent if a match is found
                break  # Break out of the inner loop if a match is found
        if intent:  # Break out of the outer loop if an intent is assigned
            break

    # If no intent is found, assign "Other"
    if intent is None:
        intent = "Other"

    return intent

# Apply the classify_intent function to populate the 'intent' column
pdf_dataset['intent'] = pdf_dataset['text'].apply(classify_intent)

# Print the updated DataFrame
print(pdf_dataset)

                                                text      intent
0  Postgraduate \nProgrammes\nDublin Business Sch...  Admissions
